In [1]:
from sklearn.metrics import mean_squared_error
from math import sqrt
from lightgbm import LGBMRegressor
!pip install --upgrade scikit-learn numpy pandas catboost 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from termcolor import colored
import plotly.express as px
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score,accuracy_score,classification_report
import warnings
warnings.filterwarnings('always') 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.8 MB 1.2 MB/s 


In [2]:
train = pd.read_csv('/content/drive/MyDrive/new_data/UmojaHack India/Train.csv')
test = pd.read_csv('/content/drive/MyDrive/new_data/UmojaHack India/Test.csv')

#feature eng

In [3]:
train['income_above_limit'] = train['income_above_limit'].map({'Above limit':1,'Below limit':0})
train['income_above_limit'].value_counts()

0    196501
1     12998
Name: income_above_limit, dtype: int64

In [4]:
test['income_above_limit'] = 999

df = pd.concat([train,test],axis=0)

df = df.reset_index(drop=True)
#nan value counts acros each row

df['t_nan_r'] = df.isnull().sum(axis=1)

In [5]:
nan_cols_drop  = []
for cols in test.columns:
    if test[cols].isna().sum()/test.shape[0] >0.8:
        nan_cols_drop.append(cols)

In [6]:
df_d = df.drop(nan_cols_drop,axis=1)

In [7]:
#for target encoding categorical features
!pip install category-encoders
from category_encoders import TargetEncoder
encoder = TargetEncoder()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 942 kB/s 


/usr/local/lib/python3.7/dist-packages/google/colab/_pip.py:83: ResourceWarning: unclosed file <_io.TextIOWrapper name='/usr/local/lib/python3.7/dist-packages/category_encoders-2.5.1.post0.dist-info/top_level.txt' mode='r' encoding='UTF-8'>
  for line in open(toplevel):
/usr/local/lib/python3.7/dist-packages/category_encoders/target_encoder.py:124: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/category_encoders/target_encoder.py:129: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  category=FutureWarning)


In [8]:
cat_cols = list(df_d.columns[df_d.dtypes=='object'])

In [ ]:
cat_cols

In [10]:
cat_cols = cat_cols[1:-1]
cat_cols

['gender',
 'education',
 'class',
 'marital_status',
 'race',
 'is_hispanic',
 'employment_commitment',
 'industry_code_main',
 'occupation_code_main',
 'household_stat',
 'household_summary',
 'under_18_family',
 'tax_status',
 'citizenship',
 'country_of_birth_own',
 'country_of_birth_father',
 'country_of_birth_mother',
 'migration_code_change_in_msa',
 'migration_prev_sunbelt',
 'migration_code_move_within_reg',
 'migration_code_change_in_reg']

In [11]:
#target encoding 
for i in cat_cols:
  Fit= encoder.fit(train[i], train['income_above_limit'])
  df_d[i] = Fit.transform(df_d[i])

In [19]:
neede_t = ['employment_stat','mig_year','occupation_code','industry_code','residence_1_year_ago']

In [18]:
df_2 = df_d.copy()

In [20]:
df_2[neede_t] = df_2[neede_t].astype('object')

In [21]:
train[neede_t] = train[neede_t].astype('object')

In [22]:
for i in neede_t:
  Fit= encoder.fit(train[i], train['income_above_limit'])
  df_2[i] = Fit.transform(df_2[i])

In [23]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299285 entries, 0 to 299284
Data columns (total 38 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   ID                              299285 non-null  object 
 1   age                             299285 non-null  int64  
 2   gender                          299285 non-null  float64
 3   education                       299285 non-null  float64
 4   class                           299285 non-null  float64
 5   marital_status                  299285 non-null  float64
 6   race                            299285 non-null  float64
 7   is_hispanic                     299285 non-null  float64
 8   employment_commitment           299285 non-null  float64
 9   employment_stat                 299285 non-null  float64
 10  wage_per_hour                   299285 non-null  int64  
 11  working_week_per_year           299285 non-null  int64  
 12  industry_code   

In [24]:
df_2.to_csv('full_d_tencoded.csv',index=False)

In [25]:
train_new = df_2[df_2.income_above_limit!=999]

test_new = df_2[df_2.income_above_limit==999]

train_new.head()

,ID,age,gender,education,class,marital_status,race,is_hispanic,employment_commitment,employment_stat,...,country_of_birth_father,country_of_birth_mother,migration_code_change_in_msa,migration_prev_sunbelt,migration_code_move_within_reg,migration_code_change_in_reg,residence_1_year_ago,importance_of_record,income_above_limit,t_nan_r
0,ID_TZ0000,79,0.025629,0.038971,0.008818,0.033095,0.067373,0.068363,0.017703,0.057825,...,0.064659,0.064833,0.066211,0.066211,0.066211,0.066211,0.065221,1779.74,0,10
1,ID_TZ0001,65,0.025629,0.038971,0.008818,0.033095,0.067373,0.068363,0.046992,0.057825,...,0.064659,0.064833,0.062120,0.061005,0.062120,0.062120,0.062120,2366.75,0,10
2,ID_TZ0002,21,0.101375,0.015337,0.200853,0.012896,0.026830,0.068363,0.046992,0.057825,...,0.064659,0.064833,0.062120,0.061005,0.062120,0.062120,0.062120,1693.42,0,7
3,ID_TZ0003,2,0.025629,0.000000,0.008818,0.012896,0.072985,0.068363,0.046992,0.057825,...,0.141085,0.150000,0.062120,0.061005,0.062120,0.062120,0.062120,1380.27,0,9
4,ID_TZ0004,70,0.101375,0.038971,0.008818,0.114176,0.067373,0.068363,0.017703,0.057825,...,0.064659,0.064833,0.066211,0.066211,0.066211,0.066211,0.065221,1580.79,0,10


In [26]:
test_new.shape

(89786, 38)

#model

In [31]:
ff=['ID','income_above_limit','kfold']
useful_feature=[c for c in test_new.columns if c not in ff] 

In [28]:
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
for fold ,(train_indicies,valid_indicies) in enumerate(kfold.split(X=train_new, y = train_new['income_above_limit'])):
  train_new.loc[valid_indicies,'kfold']=fold

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

In [29]:
sub_new = pd.DataFrame()

In [30]:
df=train_new.copy()
scores=[]
p1 = []
test_r=test_new[useful_feature]
final_test_value=[]
final_test_pr=[]
final_valid_pr={}

for fold in range(5):
  xtrain=df[df.kfold!=fold].reset_index(drop=True)
  xvalid=df[df.kfold==fold].reset_index(drop=True)
  #
  valid_ids=xvalid.ID.values.tolist()

  ytrain=xtrain['income_above_limit']
  yvalid=xvalid['income_above_limit']

  xtrain=xtrain[useful_feature]
  xvalid=xvalid[useful_feature]
 
  

  model           = CatBoostClassifier(eval_metric="F1",random_state=42,n_estimators =4000)

  model=model.fit(xtrain, ytrain ,eval_set=[(xvalid, yvalid)], verbose=20, early_stopping_rounds=500)
  preds_valid = model.predict(xvalid)
  preds_valid_prob = model.predict_proba(xvalid)[:,1]
  dd = pd.DataFrame()
  dd['prob'] = preds_valid_prob
  th = [0.40,0.35,0.3,0.29,0.28,0.25]
  
  for j in th:
    dd['last'] = np.where(dd.prob>j,1,0)
    f1 = f1_score(yvalid, dd['last'])
    p1.append(f1)
    print(fold,j,f1)

  f1 = f1_score(yvalid, preds_valid)

  print(fold,f1)
  #print("RMSLE LGBM: ", sqrt((mean_squared_error((yvalid),(valid_pred)))))

  #test_p=model.predict(test_r)
  preds_test_prob = model.predict_proba(test_r)[:,1]
  final_test_pr.append(preds_test_prob)
  #final_test_value.append(test_p)


  final_valid_pr.update(dict(zip(valid_ids,preds_valid_prob)))

  scores.append(f1)

print(np.mean(scores),np.std(scores))

final_valid_pred=pd.DataFrame.from_dict(final_valid_pr ,orient="index").reset_index()
final_valid_pred.columns=['ID','cat_p_f']
final_valid_pred.to_csv('cat_ff.csv',index=False)

#sub_new['pred_1']=np.mean(np.column_stack(final_test_value),axis=1)
sub_new['cat_p_f']=np.mean(np.column_stack(final_test_pr),axis=1)

Learning rate set to 0.061434
0:	learn: 0.2445569	test: 0.2414681	best: 0.2414681 (0)	total: 84.6ms	remaining: 5m 38s
20:	learn: 0.4184275	test: 0.4040639	best: 0.4074181 (18)	total: 879ms	remaining: 2m 46s
40:	learn: 0.4736236	test: 0.4738847	best: 0.4738847 (40)	total: 1.64s	remaining: 2m 38s
60:	learn: 0.5108759	test: 0.5144343	best: 0.5144343 (60)	total: 2.39s	remaining: 2m 34s
80:	learn: 0.5335185	test: 0.5357594	best: 0.5362025 (79)	total: 3.15s	remaining: 2m 32s
100:	learn: 0.5480763	test: 0.5518270	best: 0.5518270 (100)	total: 3.91s	remaining: 2m 30s
120:	learn: 0.5583256	test: 0.5611794	best: 0.5611794 (120)	total: 4.66s	remaining: 2m 29s
140:	learn: 0.5659014	test: 0.5659824	best: 0.5659824 (140)	total: 5.42s	remaining: 2m 28s
160:	learn: 0.5721078	test: 0.5707329	best: 0.5707329 (158)	total: 6.16s	remaining: 2m 26s
180:	learn: 0.5768040	test: 0.5748851	best: 0.5748851 (180)	total: 6.92s	remaining: 2m 26s
200:	learn: 0.5817566	test: 0.5772947	best: 0.5772947 (200)	total: 7.65

In [32]:
a = np.array(p1).reshape(5,6)
ff = np.mean(np.column_stack(a),axis=1)
ff

array([0.63058527, 0.63488943, 0.63740549, 0.63559711, 0.63418175,
       0.62925768])

In [33]:
thresh = 0.30
sub_new['last'] = np.where(sub_new['cat_p_f']>thresh,1,0)

In [34]:
subdf = pd.read_csv('/content/drive/MyDrive/new_data/UmojaHack India/SampleSubmission.csv')
subdf['ID'] = test_new['ID'].values
subdf['income_above_limit'] = sub_new['last']
subdf.head()

,ID,income_above_limit
0,ID_TZ209499,0
1,ID_TZ209500,0
2,ID_TZ209501,1
3,ID_TZ209502,0
4,ID_TZ209503,0


In [35]:
#pulic ld : 0.63390441
#private ld : 0.62930749

subdf.to_csv('first_1.csv',index=False)

In [36]:
ff=['ID','income_above_limit','kfold','t_nan_r']
useful_feature=[c for c in test_new.columns if c not in ff] 

In [37]:
df=train_new.copy()
scores=[]
p1 = []
test_r=test_new[useful_feature]
final_test_value=[]
final_test_pr=[]
final_valid_pr={}

for fold in range(5):
  xtrain=df[df.kfold!=fold].reset_index(drop=True)
  xvalid=df[df.kfold==fold].reset_index(drop=True)
  #
  valid_ids=xvalid.ID.values.tolist()

  ytrain=xtrain['income_above_limit']
  yvalid=xvalid['income_above_limit']

  xtrain=xtrain[useful_feature]
  xvalid=xvalid[useful_feature]
 
  

  model           = CatBoostClassifier(eval_metric="F1",random_state=42,n_estimators =4000)

  model=model.fit(xtrain, ytrain ,eval_set=[(xvalid, yvalid)], verbose=20, early_stopping_rounds=500)
  preds_valid = model.predict(xvalid)
  preds_valid_prob = model.predict_proba(xvalid)[:,1]
  dd = pd.DataFrame()
  dd['prob'] = preds_valid_prob
  th = [0.40,0.35,0.3,0.29,0.28,0.25]
  
  for j in th:
    dd['last'] = np.where(dd.prob>j,1,0)
    f1 = f1_score(yvalid, dd['last'])
    p1.append(f1)
    print(fold,j,f1)

  f1 = f1_score(yvalid, preds_valid)

  print(fold,f1)
  #print("RMSLE LGBM: ", sqrt((mean_squared_error((yvalid),(valid_pred)))))

  #test_p=model.predict(test_r)
  preds_test_prob = model.predict_proba(test_r)[:,1]
  final_test_pr.append(preds_test_prob)
  #final_test_value.append(test_p)


  final_valid_pr.update(dict(zip(valid_ids,preds_valid_prob)))

  scores.append(f1)

print(np.mean(scores),np.std(scores))

final_valid_pred=pd.DataFrame.from_dict(final_valid_pr ,orient="index").reset_index()
final_valid_pred.columns=['ID','cat_p_f_2']
final_valid_pred.to_csv('cat_ff.csv',index=False)

#sub_new['pred_1']=np.mean(np.column_stack(final_test_value),axis=1)
sub_new['cat_p_f_2']=np.mean(np.column_stack(final_test_pr),axis=1)

Learning rate set to 0.061434
0:	learn: 0.2911973	test: 0.2762744	best: 0.2762744 (0)	total: 42.4ms	remaining: 2m 49s
20:	learn: 0.4233868	test: 0.4076103	best: 0.4076103 (20)	total: 814ms	remaining: 2m 34s
40:	learn: 0.4695628	test: 0.4643540	best: 0.4643540 (40)	total: 1.57s	remaining: 2m 31s
60:	learn: 0.5090175	test: 0.5094044	best: 0.5094044 (60)	total: 2.32s	remaining: 2m 30s
80:	learn: 0.5331394	test: 0.5335698	best: 0.5335698 (80)	total: 3.08s	remaining: 2m 29s
100:	learn: 0.5464986	test: 0.5465000	best: 0.5465000 (100)	total: 3.82s	remaining: 2m 27s
120:	learn: 0.5564960	test: 0.5571182	best: 0.5571182 (118)	total: 4.57s	remaining: 2m 26s
140:	learn: 0.5636464	test: 0.5623927	best: 0.5625307 (139)	total: 5.3s	remaining: 2m 25s
160:	learn: 0.5702257	test: 0.5660745	best: 0.5673171 (156)	total: 6.07s	remaining: 2m 24s
180:	learn: 0.5765163	test: 0.5703883	best: 0.5708738 (175)	total: 6.8s	remaining: 2m 23s
200:	learn: 0.5823643	test: 0.5759264	best: 0.5759264 (200)	total: 7.53s	

In [38]:
a = np.array(p1).reshape(5,6)
ff = np.mean(np.column_stack(a),axis=1)
ff

array([0.62891064, 0.63363436, 0.63499468, 0.63512854, 0.63456665,
       0.63053313])

In [39]:
#take the threshold value according to the ff list values which is the mean of each th for each fold
thresh = 0.29
sub_new['last'] = np.where(sub_new['cat_p_f_2']>thresh,1,0)

In [44]:
sub_new['last'].value_counts()

0    84203
1     5583
Name: last, dtype: int64

In [40]:
subdf = pd.read_csv('/content/drive/MyDrive/new_data/UmojaHack India/SampleSubmission.csv')
subdf['ID'] = test_new['ID'].values
subdf['income_above_limit'] = sub_new['last']
subdf.head()

,ID,income_above_limit
0,ID_TZ209499,0
1,ID_TZ209500,0
2,ID_TZ209501,1
3,ID_TZ209502,0
4,ID_TZ209503,0


the output taken for scoring

In [41]:
#pulic ld : 0.6333038
#private ld : 0.6296659

subdf.to_csv('first_2.csv',index=False)

In [42]:
#thresh = 0.31
#sub_new['last'] = np.where(sub_new['cat_p_f_2']>thresh,1,0)

In [ ]:
#subdf = pd.read_csv('/content/drive/MyDrive/new_data/UmojaHack India/SampleSubmission.csv')
#subdf['ID'] = test_new['ID'].values
#subdf['income_above_limit'] = sub_new['last']
#subdf.head()

In [45]:


#subdf.to_csv('first_3.csv',index=False)

for bettter score we can do feature selection and adjusting the learning rate.

plz mail we if any change need any explantion needed for the notebook

In [ ]:
df=train_new.copy()
scores=[]
p1 = []
test_r=test_new[useful_feature]
final_test_value=[]
final_test_pr=[]
final_valid_pr={}

for fold in range(5):
  xtrain=df[df.kfold!=fold].reset_index(drop=True)
  xvalid=df[df.kfold==fold].reset_index(drop=True)
  #
  valid_ids=xvalid.ID.values.tolist()

  ytrain=xtrain['income_above_limit']
  yvalid=xvalid['income_above_limit']

  xtrain=xtrain[useful_feature]
  xvalid=xvalid[useful_feature]
 
  

  model           = CatBoostClassifier(eval_metric="F1",random_state=42,n_estimators =4000)

  model=model.fit(xtrain, ytrain ,eval_set=[(xvalid, yvalid)], verbose=20, early_stopping_rounds=500)
  preds_valid = model.predict(xvalid)
  preds_valid_prob = model.predict_proba(xvalid)[:,1]
  dd = pd.DataFrame()
  dd['prob'] = preds_valid_prob
  th = [0.40,0.35,0.3,0.29,0.28,0.25]
  
  for j in th:
    dd['last'] = np.where(dd.prob>j,1,0)
    f1 = f1_score(yvalid, dd['last'])
    p1.append(f1)
    print(fold,j,f1)

  f1 = f1_score(yvalid, preds_valid)

  print(fold,f1)
  #print("RMSLE LGBM: ", sqrt((mean_squared_error((yvalid),(valid_pred)))))

  #test_p=model.predict(test_r)
  preds_test_prob = model.predict_proba(test_r)[:,1]
  final_test_pr.append(preds_test_prob)
  #final_test_value.append(test_p)


  final_valid_pr.update(dict(zip(valid_ids,preds_valid_prob)))

  scores.append(f1)

print(np.mean(scores),np.std(scores))

final_valid_pred=pd.DataFrame.from_dict(final_valid_pr ,orient="index").reset_index()
final_valid_pred.columns=['ID','cat_p_f_2']
final_valid_pred.to_csv('cat_ff.csv',index=False)

#sub_new['pred_1']=np.mean(np.column_stack(final_test_value),axis=1)
sub_new['cat_p_f_2']=np.mean(np.column_stack(final_test_pr),axis=1)